In [42]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE


from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from scipy.stats import boxcox
from sklearn import tree


s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'train.csv'
file_key2= 'test.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')
bucket_object2= bucket.Object(file_key2)
file_object2 = bucket_object2.get()
file_content_stream2 = file_object2.get('Body')

#reading the datefile
train = pd.read_csv(file_content_stream)
test = pd.read_csv(file_content_stream2)

In [43]:
#Engineering variables
#Variable 1: Box Cox on scannedLineItemsPerSecond
transformed_trustLevel= boxcox(train['trustLevel'])
train['trustLevel'] = transformed_trustLevel[0]

transformed_trustLevel= boxcox(test['trustLevel'])
test['trustLevel'] = transformed_trustLevel[0]

In [44]:
##Engineering variables
#2: Log base 10
train['log_grandTotal'] = np.log10(train['grandTotal'])

test['log_grandTotal'] = np.log10(test['grandTotal'])

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [45]:
##Engineering variables
#3: x^2
train['squared_valuePerSecond']= np.square(train['valuePerSecond'])

test['squared_valuePerSecond']= np.square(test['valuePerSecond'])

In [46]:
##Engineering variables
#4: 1/x
train['1_scannedLineItemsPerSecond']= 1/(train['scannedLineItemsPerSecond'])

test['1_scannedLineItemsPerSecond']= 1/(test['scannedLineItemsPerSecond'])

In [47]:
##Engineering variables
#5: x^3
train['cubed_lineItemVoidsPerPosition']= np.power(train['lineItemVoidsPerPosition'], 3)

test['cubed_lineItemVoidsPerPosition']= np.power(test['lineItemVoidsPerPosition'], 3)


In [48]:
##Engineering variables
#6: Natural Log
train['NL_totalScanTimeInSeconds']= np.log(train['totalScanTimeInSeconds'])

test['NL_totalScanTimeInSeconds']= np.log(test['totalScanTimeInSeconds'])


In [49]:
##Engineering variables
#7: Differencce
train['VPS_SLIPS']= train['valuePerSecond'] - train['scannedLineItemsPerSecond']

test['VPS_SLIPS']= train['valuePerSecond'] - train['scannedLineItemsPerSecond']

In [50]:
#Engineering interactions based on feature importance
train['interaction_1']= train['trustLevel'] * train['totalScanTimeInSeconds']
train['interaction_2']= train['trustLevel'] * train['lineItemsVoids']
train['interaction_3']= train['totalScanTimeInSeconds'] * train['lineItemsVoids']


test['interaction_1']= test['trustLevel'] * test['totalScanTimeInSeconds']
test['interaction_2']= test['trustLevel'] * test['lineItemVoids']
test['interaction_3']= test['totalScanTimeInSeconds'] * test['lineItemVoids']

In [51]:
#engineering features from the decision tree model
train['interaction_4']=np.where((train['scannedLineItemsPerSecond'] <= .012) &
                                   (train['totalScanTimeInSeconds'] > 299.5), 1, 0)
                                 
train['interaction_5']=np.where(((train['scannedLineItemsPerSecond'] <= .015) &
                                   (train['totalScanTimeInSeconds'] <= 1829.5) &
                                (train['scannedLineItemsPerSecond'] <= .012)), 1, 0)
        
train['interaction_6']=np.where(((train['scannedLineItemsPerSecond'] <= .015) &
                                   (train['totalScanTimeInSeconds'] <= 1829.5) &
                                (train['valuePerSecond'] <= .115)), 1, 0)


#engineering features from the decisoon tree model
test['interaction_4']=np.where((test['scannedLineItemsPerSecond'] <= .012) &
                                   (test['totalScanTimeInSeconds'] > 299.5), 1, 0)
                                 
test['interaction_5']=np.where(((test['scannedLineItemsPerSecond'] <= .015) &
                                   (test['totalScanTimeInSeconds'] <= 1829.5) &
                                (test['scannedLineItemsPerSecond'] <= .012)), 1, 0)
        
test['interaction_6']=np.where(((test['scannedLineItemsPerSecond'] <= .015) &
                                   (test['totalScanTimeInSeconds'] <= 1829.5) &
                                (test['valuePerSecond'] <= .115)), 1, 0)

In [60]:
lr= list()
rf= list()
ada= list()

for i in range(0,100):
    
    #Defining input and target variables
    x= train.drop(columns=['fraud'], axis=1)
    y= train['fraud']

    #Splitting the data
    x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify=y)
    
    
    #running RFE with logistic regression
    lr_rfe = RFE(estimator = LogisticRegression(), n_features_to_select = 5).fit(x_train, y_train)
    #appending rankings
    lr.append(lr_rfe.support_)
    
    
    #running RFE with Random forest model
    rf_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), n_features_to_select = 5).fit(x_train, y_train)
    #appending rankings
    rf.append(rf_rfe.support_)
    
    
    #running RFE with Random forest model
    ada_rfe = RFE(estimator = AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3), n_estimators=500, learning_rate=.01), n_features_to_select = 5).fit(x_train, y_train)
    #appending rankings
    ada.append(ada_rfe.support_)
    

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

In [61]:
lr= pd.DataFrame(lr)
lr.columns=x_train.columns
100*lr.apply(np.sum, axis=0)/lr.shape[0]

trustLevel                        100.0
totalScanTimeInSeconds              0.0
grandTotal                          0.0
lineItemsVoids                      1.0
scansWithoutRegistration            0.0
quanitityModification               0.0
scannedLineItemsPerSecond           6.0
valuePerSecond                      0.0
lineItemVoidsPerPosition           24.0
log_grandTotal                      0.0
squared_valuePerSecond              0.0
1_scannedLineItemsPerSecond         0.0
cubed_lineItemVoidsPerPosition     94.0
NL_totalScanTimeInSeconds         100.0
VPS_SLIPS                          70.0
interaction_1                       0.0
interaction_2                       0.0
interaction_3                       0.0
interaction_4                       6.0
interaction_5                       9.0
interaction_6                      90.0
dtype: float64

In [62]:
rf= pd.DataFrame(rf)
rf.columns=x_train.columns
100*rf.apply(np.sum, axis=0)/rf.shape[0]

trustLevel                        100.0
totalScanTimeInSeconds              8.0
grandTotal                          0.0
lineItemsVoids                      0.0
scansWithoutRegistration            0.0
quanitityModification               0.0
scannedLineItemsPerSecond          80.0
valuePerSecond                      0.0
lineItemVoidsPerPosition           14.0
log_grandTotal                      0.0
squared_valuePerSecond              0.0
1_scannedLineItemsPerSecond        89.0
cubed_lineItemVoidsPerPosition     14.0
NL_totalScanTimeInSeconds           7.0
VPS_SLIPS                           0.0
interaction_1                     100.0
interaction_2                       0.0
interaction_3                      88.0
interaction_4                       0.0
interaction_5                       0.0
interaction_6                       0.0
dtype: float64

In [66]:
ada= pd.DataFrame(ada)
ada.columns=x_train.columns
100*ada.apply(np.sum, axis=0)/ada.shape[0]

trustLevel                         2.0
totalScanTimeInSeconds            72.0
grandTotal                         8.0
lineItemsVoids                     2.0
scansWithoutRegistration          44.0
quanitityModification              0.0
scannedLineItemsPerSecond         51.0
valuePerSecond                     6.0
lineItemVoidsPerPosition           1.0
log_grandTotal                    13.0
squared_valuePerSecond             3.0
1_scannedLineItemsPerSecond       57.0
cubed_lineItemVoidsPerPosition     5.0
NL_totalScanTimeInSeconds         60.0
VPS_SLIPS                          5.0
interaction_1                     77.0
interaction_2                      0.0
interaction_3                     94.0
interaction_4                      0.0
interaction_5                      0.0
interaction_6                      0.0
dtype: float64

In [76]:
models= pd.merge(lr, rf, ada, how='inner', on= list(x_train.columns))

TypeError: merge() got multiple values for argument 'how'

In [77]:
models = pd.merge(lr, pd.merge(rf, ada, on=list(x_train.columns)), on=list(x_train.columns))

In [78]:
models

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,log_grandTotal,...,1_scannedLineItemsPerSecond,cubed_lineItemVoidsPerPosition,NL_totalScanTimeInSeconds,VPS_SLIPS,interaction_1,interaction_2,interaction_3,interaction_4,interaction_5,interaction_6


In [73]:
models = [lr, rf, ada]
result = pd.concat(models)

In [79]:
models = pd.concat([lr, rf, ada])

In [80]:
models

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,log_grandTotal,...,1_scannedLineItemsPerSecond,cubed_lineItemVoidsPerPosition,NL_totalScanTimeInSeconds,VPS_SLIPS,interaction_1,interaction_2,interaction_3,interaction_4,interaction_5,interaction_6
0,True,False,False,False,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,True
1,True,False,False,False,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,True
2,True,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,True,False,True
3,True,False,False,False,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,True
4,True,False,False,False,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,False,False,False,True,False,False,False,False,False,...,True,False,True,False,True,False,True,False,False,False
96,False,True,False,False,False,False,False,False,False,False,...,True,False,True,False,True,False,True,False,False,False
97,False,False,False,False,True,False,True,False,False,False,...,False,False,True,False,True,False,True,False,False,False
98,False,False,False,False,False,False,True,False,False,True,...,False,False,True,False,True,False,True,False,False,False


In [85]:
column_means = models.mean(axis=0)
column_means*100

trustLevel                        67.333333
totalScanTimeInSeconds            26.666667
grandTotal                         2.666667
lineItemsVoids                     1.000000
scansWithoutRegistration          14.666667
quanitityModification              0.000000
scannedLineItemsPerSecond         45.666667
valuePerSecond                     2.000000
lineItemVoidsPerPosition          13.000000
log_grandTotal                     4.333333
squared_valuePerSecond             1.000000
1_scannedLineItemsPerSecond       48.666667
cubed_lineItemVoidsPerPosition    37.666667
NL_totalScanTimeInSeconds         55.666667
VPS_SLIPS                         25.000000
interaction_1                     59.000000
interaction_2                      0.000000
interaction_3                     60.666667
interaction_4                      2.000000
interaction_5                      3.000000
interaction_6                     30.000000
dtype: float64

In [ ]:
#Based on my data splits and models, I would rank the variables in this order:
#1- trustLevel                        67.333333
#9- totalScanTimeInSeconds            26.666667
#15- grandTotal                         2.666667
#1/19- lineItemsVoids                     1.000000
#11- scansWithoutRegistration          14.666667
#20/21- quanitityModification              0.000000
#6- scannedLineItemsPerSecond         45.666667
#16/17- valuePerSecond                     2.000000
#12- lineItemVoidsPerPosition          13.000000
#13- log_grandTotal                     4.333333
#18/19- squared_valuePerSecond             1.000000
#5- 1_scannedLineItemsPerSecond       48.666667
#7- cubed_lineItemVoidsPerPosition    37.666667
#4- NL_totalScanTimeInSeconds         55.666667
#10- VPS_SLIPS                         25.000000
#3- interaction_1                     59.000000
#20/21- interaction_2                      0.000000
#2- interaction_3                     60.666667
#16/17- interaction_4                      2.000000
#14- interaction_5                      3.000000
#8- interaction_6                     30.000000

#Note: Higher number means they were more significant more often!!

#So, if I were to build a model with the top 5 variables, I would implement trustLevel, interaction_3, interaction_1, NL_totalScanTimeInSeconds and 1_scannedLineItemsPerSecond